In [87]:
"""
Preamble for most code and jupyter notebooks
@author: tobinsouth
@notebook date: May 19, 2023
"""

import numpy as np, pandas as pd, matplotlib.pyplot as plt, matplotlib as mpl, seaborn as sns
import math, string, re, pickle, json, os, sys, datetime, itertools, glob
from collections import Counter
from tqdm import tqdm
from tueplots import bundles


In [88]:
ghns = ['geohash9','geohash8','geohash7','geohash6','geohash5','geohash4','geohash3']

## Load in Cuebiq PSI Results

In [101]:
def load_zip_to_zip_psi(file):
    zip_to_zip_overlap_means_df = pd.read_csv(file)
    zip_to_zip_overlap_means_df['zip1'] = zip_to_zip_overlap_means_df['zip1'].astype(str).str.zfill(5)
    zip_to_zip_overlap_means_df['zip2'] = zip_to_zip_overlap_means_df['zip2'].astype(str).str.zfill(5)
    # Log geohashs
    for ghn in ghns:
        zip_to_zip_overlap_means_df['log_'+ghn] = np.log1p(zip_to_zip_overlap_means_df[ghn])
    return zip_to_zip_overlap_means_df

zip2zipPIS = {}
for file in glob.glob('results/zip_to_zip_overlap_means_df*.csv'):
    zip2zipPIS[file.split(".")[0][-5:]] = load_zip_to_zip_psi(file)


In [102]:
all_zips_with_data = set()
for df in zip2zipPIS.values():
    all_zips_with_data = all_zips_with_data | set(df['zip1']) | set(df['zip2'])

## Load in FB Social Connectendness Index

In [91]:
!cd data/fb-sci && unzip us-zip-code-us-zip-code-fb-social-connectedness-index-october-2021.zip && A

Archive:  us-zip-code-us-zip-code-fb-social-connectedness-index-october-2021.zip
replace zcta_zcta_shard5.tsv? [y]es, [n]o, [A]ll, [N]one, [r]ename: ^C


In [92]:
# Loop through the zipfile `us-zip-code-us-zip-code-fb-social-connectedness-index-october-2021.zip` to access each file titled 'zcta_zcta_shardN.tsv`. 
# Each file contains the social connectedness index for zipcodes in the US. Keep only those zipcodes that are in the `all_zips_with_data` set.

# Read in the social connectedness index for each zipcode
all_social_connectedness = []
import glob
for file in tqdm(glob.glob('data/fb-sci/zcta_zcta_shard*.tsv')):
    social_connectedness = pd.read_csv(file, sep='\t')
    social_connectedness['user_loc'] = social_connectedness['user_loc'].astype(str).str.zfill(5)
    social_connectedness['fr_loc'] = social_connectedness['fr_loc'].astype(str).str.zfill(5)    
    social_connectedness = social_connectedness[social_connectedness['user_loc'].isin(all_zips_with_data) & social_connectedness['fr_loc'].isin(all_zips_with_data)]

    all_social_connectedness.append(social_connectedness)
social_connectedness = pd.concat([sc for sc in all_social_connectedness if len(sc) > 0])
social_connectedness.rename(columns={'user_loc': 'zip1', 'fr_loc': 'zip2'}, inplace=True)

100%|██████████| 10/10 [09:28<00:00, 56.88s/it]


In [93]:
social_connectedness['log_sci'] = np.log1p(social_connectedness['scaled_sci'])
# social_connectedness = social_connectedness[social_connectedness['log_sci'] != 0]

In [94]:
social_connectedness

,zip1,zip2,scaled_sci,log_sci
11200201,53104,01431,1,0.693147
11200202,53104,01432,1,0.693147
11200203,53104,01434,1,0.693147
11200206,53104,01450,4670,8.449128
11200210,53104,01460,2041,7.621685
...,...,...,...,...
16582697,93591,93550,1399346,14.151516
16582698,93591,93551,1163304,13.966776
16582699,93591,93552,1693713,14.342434
16582700,93591,93553,3493403,15.066387


## Gravity Model Baseline

In [95]:
import math
import pandas as pd

def haversine_distance(lat1, lon1, lat2, lon2):
    """ Calculate the great circle distance between two points. More accurate than euclidean distance for Earth locations."""
    R = 6371  # Earth's radius in kilometers
    dlat = math.radians(lat2 - lat1)
    dlon = math.radians(lon2 - lon1)
    a = (math.sin(dlat / 2) * math.sin(dlat / 2) +
         math.cos(math.radians(lat1)) * math.cos(math.radians(lat2)) *
         math.sin(dlon / 2) * math.sin(dlon / 2))
    c = 2 * math.atan2(math.sqrt(a), math.sqrt(1 - a))
    return R * c

def load_zipcode_coordinates(url):
    df = pd.read_csv(url)
    df['ZIP'] = df['ZIP'].astype(str).str.zfill(5)
    return {row.ZIP: (row.LAT, row.LNG) for _, row in df.iterrows()}

url = "https://gist.githubusercontent.com/erichurst/7882666/raw/5bdc46db47d9515269ab12ed6fb2850377fd869e/US%2520Zip%2520Codes%2520from%25202013%2520Government%2520Data"
zipcode_coordinates = load_zipcode_coordinates(url)


# Calculate distances
zip_to_zip_distance = []
for zip1, zip2 in social_connectedness[['zip1','zip2']].itertuples(index=False):
    if zip1 is not np.nan and zip2 is not np.nan:
        distance = haversine_distance(zipcode_coordinates[zip1][0], zipcode_coordinates[zip1][1], zipcode_coordinates[zip2][0], zipcode_coordinates[zip2][1])
        zip_to_zip_distance.append([zip1, zip2, distance])

zip_to_zip_distance_df = pd.DataFrame(zip_to_zip_distance, columns=['zip1', 'zip2', 'distance'])
zip_to_zip_distance_df['gravity'] = zip_to_zip_distance_df['distance']**(-2)
zip_to_zip_distance_df['gravity1.5'] = zip_to_zip_distance_df['distance']**(-1.5)
zip_to_zip_distance_df['gravity1'] = zip_to_zip_distance_df['distance']**(-1)
social_connectedness_distance = social_connectedness.merge(zip_to_zip_distance_df, left_on=['zip1', 'zip2'], right_on=['zip1', 'zip2'], how='left')

## Combine Gravity + FB SCI w/ Cuebiq PSI

In [103]:
all_social_connectedness_overlap = {}
for regioncode, zip_to_zip_overlap_means_df in zip2zipPIS.items():
    # Join dataframes
    social_connectedness_overlap = social_connectedness_distance.merge(zip_to_zip_overlap_means_df, left_on=['zip1', 'zip2'], right_on=['zip1', 'zip2'], how='right')

    all_social_connectedness_overlap[regioncode] = social_connectedness_overlap

In [104]:
len(social_connectedness), [len(sc) for sc in all_social_connectedness_overlap.values()]

(4866436, [39060, 463203, 68635, 80200, 37675])

In [105]:
[len(sc) for sc in zip2zipPIS.values()]

[39060, 463203, 68635, 80200, 37675]

In [85]:
# # Sanity check
# for regioncode, social_connectedness_overlap in all_social_connectedness_overlap.items():
#     print(regioncode)
#     plt.imshow(social_connectedness_overlap.corr('pearson'))
#     plt.show()

#     social_connectedness_overlap.plot.scatter('log_geohash7', 'log_sci', alpha=0.7, s=1)
#     plt.show()

In [112]:
for regioncode, social_connectedness_overlap in all_social_connectedness_overlap.items():
    social_connectedness_overlap.to_csv(f'results/all_PSI_SCI_gravity_{regioncode}.csv.gz', compression='gzip')